### Welcome to CellCounter!

This is a jupyter notebook that enables a user to automatically detect cells labeled with fluorescent markers after immunohistochemistry. By following the markdown documentation within this notebook, you will be able to (hopefully!) extract the number of cells that are positive for your marker of interest. The goal of CellCounter is to be user friendly and allow the user to adjust parameters to more accurately extract cells. Please feel free to open any GitHub issues or requests for new functions.

This notebook supports multiple ROI drawing - if you have a picture of an entire brain slice, you would be able to draw multiple regions of interest and count cells within each ROI separately.

Currently under development:
1. Manually counting cells with circular ROIs.
2. Save preprocessed images.
3. Watershed method.

In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
import xarray as xr
import json 
from os.path import join as pjoin
import plotly.express as px
from dash import Dash, dcc, callback, html, Output, Input, no_update

from preprocessing import (
    load_image,
    denoise,
    remove_background,
    calculate_threshold,
    detect_cells,
    contour_selection
)

from visualizations import (
    extract_roi,
    visualize_intensity_histogram,
    crop_image_to_roi
)

def visualize_image(image, binary_string=True, create_roi=True, port=9091, rois=None, **kwargs):
    fig = px.imshow(image, binary_string=binary_string)
    fig.update_layout(modebar_add=['drawrect', 'drawclosedpath', 'eraseshape'], 
                      dragmode='pan', 
                      newshape=dict(line_color='darkorchid'),
                      **kwargs)
    if rois is not None:
        for _, row in rois.iterrows():
            fig.add_shape(type='rect',
                x0=int(row['x0']), y0=int(row['y0']), x1=int(row['x1']), y1=int(row['y1']),
                line=dict(color='darkorchid'),
            )
        

    if create_roi:
        app = Dash(__name__)
        app.layout = html.Div(children=[
            dcc.Graph(id='plot', 
                      figure=fig, 
                      config={'scrollZoom': True}),
            html.Div([
                    html.Pre(id='selected-data'),
                ]),
        ])

        @callback(
            Output('selected-data', 'children'),
            Input('plot', 'relayoutData'),
            prevent_initial_call=True)
        def display_selected_data(relayoutData):
            if 'shapes' in relayoutData:
                global relayoutdata
                relayoutdata = json.dumps(relayoutData['shapes'])
                return relayoutdata
            else:
                return no_update
        
        if port is not None:
            app.run_server(port=port)
        else:
            app.run_server(mode='inline')
    else:
        return fig.show(config={'scrollZoom': True})

### Description of settings.

PyCellCounter has a few parameters that are important to be familiar with. They include:
1. pattern - a string that is part of the name of your file of interest. For example, if your image is test_tdTomato.tiff, pattern can be tdTomato, test, etc.
2. image_path - path to where your raw image data is.
3. output_path - path to where your cell count data with contours will be stored.
4. cell_diameter - average diameter of cells that will be used for median filtering to remove noise and tophat filtering to subtract the background.
5. min_area - a cell's minimum contour area to be included in your counted cells.
6. max_area - a cell's maximum contour area to be included in your counted cells.
7. avg_cell_area - average contour area to be included in your counted cells.
8. connected_cell_area - overlapping contour area to separate cells that will be included in your counted cells.
9. config - config for graphs. ScrollZoom allows you to zoom into a graph using your mouse wheel.
10. draw_roi - **Very important**. If True, you can draw ROIs in the first visualization step.
12. visualize_only_cells - **Very important**. If True, at the end of the notebook you will only visualize the contours of the cells.
13. contour_path - **Very important**. By default None. If None, you will be imaging the contours from the notebook. Otherwise you will visualize the contours that you load in through the specified path. Can be used if you want to visualize previously saved results and load previous ROIs.

In [ ]:
## Settings
mouse = 'mcr03' 
channel_name = 'ch2'
ieg = 'cfos'
pattern = 'cfos'
project_dir = 'CircleTrack_Recall'
experiment_dir = 'Recall1'
image_path = f'../CircleTrack/{project_dir}/{experiment_dir}/brain_slices/{mouse}/{ieg}/{channel_name}'
output_path = f'../CircleTrack/{project_dir}/{experiment_dir}/brain_slices/{mouse}/output/Austin'

if not os.path.exists(output_path):
    os.makedirs(output_path)
    
## Preprocessing parameters
cell_diameter = 10
median_params = {'ksize': cell_diameter // 2}
tophat_struct = cell_diameter * 7
min_area = 600
max_area = 2500
avg_cell_area = 1000
connected_cell_area = 1000

## Key image parameters
draw_roi = False ## If True you can draw roi(s) in the first visualization step
crop_image = False ## If True, you can draw an ROI and then use that ROI to crop your analysis to just that ROI
visualize_only_cells = False ## If False, will plot contours on top of binarized image
contour_path = output_path # Can either be None or the path to where your previously counted cell data is stored

## Visualization parameters
contour_color = 100
height = 800
width = 800
config = {'scrollZoom': True}

In [ ]:
## Load image of interest. The variable can be named whatever you like.
image = load_image(image_path, pattern=pattern, varr_name=f'{pattern}_channel')
image

In [ ]:
## Visualize image and draw roi(s) for further processing if roi_path is None
if contour_path is not None and draw_roi:
    raise Warning('Caution! You are loading a previously created roi while drawing a new one!')
elif contour_path is not None and not draw_roi:
    print('Loading previously drawn ROI...')
    rois = pd.DataFrame()
    for idx, file in enumerate(os.listdir(contour_path)):
        previous_contours = xr.open_dataarray(pjoin(contour_path, file))
        rois.loc[idx, 'roi'] = 0
        rois.loc[idx, 'x0'] = previous_contours.attrs['x0']
        rois.loc[idx, 'x1'] = previous_contours.attrs['x1']
        rois.loc[idx, 'y0'] = previous_contours.attrs['y0']
        rois.loc[idx, 'y1'] = previous_contours.attrs['y1']

    visualize_image(image, create_roi=draw_roi, rois=rois, height=height, width=width)
else:
    print('Enabling ROI drawing...')
    visualize_image(image, create_roi=draw_roi, height=height, width=width)

In [ ]:
## Subset image to the ROI(s) and denoise
im = image.copy()
if draw_roi and crop_image:
    rois = extract_roi(relayoutdata, shape_numbers=[0])
    im = crop_image_to_roi(im, rois)
elif draw_roi and not crop_image:
    roi_data = json.loads(relayoutdata)
    shape_numbers =  np.arange(0, len(roi_data))
    rois = extract_roi(relayoutdata, shape_numbers=shape_numbers)

if contour_path is not None and crop_image:
    for idx, file in enumerate(os.listdir(contour_path)):
        previous_contours = xr.open_dataarray(pjoin(contour_path, file))
        rois = pd.DataFrame({'roi': 0, 'x0': previous_contours.attrs['x0'], 'x1': previous_contours.attrs['x1'],
                             'y0': previous_contours.attrs['y0'], 'y1': previous_contours.attrs['y1']})
        im = crop_image_to_roi(im, rois)

im = denoise(im, method='median', **median_params)

if not crop_image:
    visualize_image(im, create_roi=False, height=height, width=width, rois=rois)
else:
    visualize_image(im, create_roi=False, height=height, width=width)

In [ ]:
## Remove background and visualize background removal
im = remove_background(im, method='tophat', wnd=tophat_struct)
if not crop_image:
    visualize_image(im, binary_string=True, create_roi=False, height=height, width=width, rois=rois)
else:
    visualize_image(im, binary_string=False, create_roi=False, height=height, width=width)

In [ ]:
## Visualize a good threshold for binarization
potential_threshold = 10 ## can edit based upon histogram
threshold = calculate_threshold(im, thresh_type='otsu')
fig = visualize_intensity_histogram(im, bins=255, range=[0, 255], density=True)
fig.add_vline(x=threshold, line_width=1, line_dash='dash', line_color='darkorchid', opacity=1)
fig.add_vline(x=potential_threshold, line_width=1, line_dash='dash', line_color='red', opacity=1)
fig.show(config=config)

Edit the binary_threshold value below based upon your visualization above. You can visualize the results of your binarization below.

In [ ]:
## Binarize image
binary_threshold = threshold ## edit value, either threshold or potential_threshold
image_binary = im.where(im >= binary_threshold, other=0)
image_binary = image_binary.where(image_binary < binary_threshold, other=255)
if not crop_image:
    visualize_image(image_binary, create_roi=False, height=height, width=width, rois=rois)
else:
    visualize_image(image_binary, create_roi=False, height=height, width=width)

The parameters below are included at the beginning and end of the notebook to ensure you are aware of their values and give you the opportunity to change them here. They will be saved along with the ROIs.

In [ ]:
## Edit parameters if needed.
min_area = 500
max_area = 2000
avg_cell_area = 700
connected_cell_area = 500

In [ ]:
## Detect cells and save results to output_path
for idx, row in rois.iterrows():
    if not crop_image:
        im_subset = image_binary[int(row['y0']):int(row['y1']), int(row['x0']):int(row['x1'])]
    else:
        im_subset = image_binary.copy()
    
    contours, hierarchy = cv2.findContours(im_subset.values, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    num_cells, cell_contours = detect_cells(im_subset, 
                                            contours,
                                            contour_color=contour_color,
                                            minimum_area=min_area, 
                                            maximum_area=max_area, 
                                            average_cell_area=avg_cell_area, 
                                            connected_cell_area=connected_cell_area)

    contour_ar = xr.DataArray(cell_contours,
                            dims=['height', 'width'],
                            name=f"roi_{row['roi']+1}",
                            attrs=dict(
                            x0=int(row['x0']),
                            x1=int(row['x1']),
                            y0=int(row['y0']),
                            y1=int(row['y1']),
                            num_cells=num_cells,
                            min_area=min_area,
                            max_area=max_area,
                            avg_cell_area=avg_cell_area,
                            connected_cell_area=connected_cell_area,
                            file_name=pattern
                            )
                        )
    contour_ar.astype(np.int8).to_netcdf(path=pjoin(output_path, f'{pattern}_{channel_name}_cell_contours_{contour_ar.name}.nc'))

In [ ]:
## Visualize your results
plot_data, num_contours = contour_selection(contour_ar=contour_ar, contour_color=contour_color, visualize_only_cells=visualize_only_cells, contour_path=None)
visualize_image(plot_data, create_roi=False, height=height, width=width, title=dict(text=f'Number of cells: {num_contours}'))